In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from itertools import permutations
from collections import Counter
from math import ceil

import pingouin as pg

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor 
#from statsmodels.tools.tools import add_constant

from scipy.stats import ks_2samp
from scipy.stats import rankdata
from scipy.special import softmax
from scipy.stats import zscore

import os
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import PolynomialFeatures

from stargazer.stargazer import Stargazer



## Utils

In [4]:
def get_variables(from_, to_, incl_rect=True):
    
    years = [y for y in range(from_, to_+1)]
    
    variables = [f"frq_{y}" for y in years]
    variables.extend([f"fpm_{y}" for y in years])
    variables.extend([f"gch_{y}:{y+1}" for y in years[:-1]])
    if incl_rect:
        variables.extend([f"rch_{y}:{y+1}" for y in years[:-1]])
    
    years = [str(y) for y in years]
    
    return variables, years

In [7]:
# Selection of terms in original dataset
terms = [
#'N1_kulturberikare',
'V1_berika',
#'N1_berikare',
'N1_globalist',
#'V1_kulturberika',
'N1_återvandring',
#'V1_återvandra',
#'A1_globalistisk',
'N1_förortsgäng',
]

In [8]:
def get_df(
    path,            # path to csv-file
    variables,       # variables to include
    terms,           # terms to include
    min_freq=10,     # int for min. freq.
    force=False,     # provide bool for sub-selection using predefined list
    external=None,   # provide dict for sub-selction if force=True
    drop_frq=False):  # drop frq after sub_selection, or not
    
    df = pd.read_csv(Path(path), sep=";", index_col = 0)
    terms = [t for t in terms if t in df.index]
    #print(terms)
    df = df[variables]
    df = df.loc[terms]
    
    if min_freq != None:
        years = [col.split("_")[-1] for col in df.columns if col.startswith("frq_")]
        years.sort()
        
        for dwe in df.index:
            for year in years:
                
                if force and dwe in external.keys():
                    if int(year) in external[dwe]: 
                        if df.loc[dwe][f"frq_{year}"] < 10:
                            print("For", dwe, "in", year, "change", df.loc[dwe][f"frq_{year}"], "to", 10)
                            df.at[dwe, f"frq_{year}"] = 10
                        else:
                            continue
                    else:
                        df.at[dwe, f"fpm_{year}"] = np.nan
                        if year == years[-1]:
                            df.at[dwe, f"gch_{int(year)-1}:{year}"] = np.nan
                            df.at[dwe, f"rch_{int(year)-1}:{year}"] = np.nan
                        else:
                            df.at[dwe, f"gch_{year}:{int(year)+1}"] = np.nan
                            df.at[dwe, f"rch_{year}:{int(year)+1}"] = np.nan  
                else:
                    if df.loc[dwe][f"frq_{year}"] < min_freq: 
                        df.at[dwe, f"fpm_{year}"] = np.nan

                        if year == years[-1]:
                            df.at[dwe, f"gch_{int(year)-1}:{year}"] = np.nan
                            df.at[dwe, f"rch_{int(year)-1}:{year}"] = np.nan
                        else:
                            df.at[dwe, f"gch_{year}:{int(year)+1}"] = np.nan
                            df.at[dwe, f"rch_{year}:{int(year)+1}"] = np.nan                        
    
    if drop_frq:
        df.drop([col for col in df.columns if col.startswith("frq_")], axis=1, inplace=True)

    return df

In [9]:
def get_iod(path, ref_path, variables, terms, min_freq=10):
    
    df = pd.read_csv(path, index_col=0)
        
    if type(ref_path) != pd.core.frame.DataFrame:
        ref = get_df(ref_path, variables, terms, min_freq = None, drop_frq=False)
    else:
        ref = ref_path
    
    years = [col.split("_")[-1] for col in ref.columns if col.startswith("frq_")]
        
    if min_freq != None:
        for dwe in ref.index:
            for year in years:
                if ref.loc[dwe][f"frq_{year}"] < min_freq:
                    df.loc[df["DWE"] == dwe, year] = np.nan    
                    
    return df


In [12]:
def add_iod_dif(df, years):
    transitions = [(y, str(int(y)+1)) for y in years[:-1]]
    df = df.copy()
    for yi, yj in transitions:
        df[f"{yi}:{yj}"] = df[f"{yj}"] - df[f"{yi}"]

    return df

## COLLECT FLASHBACK DATA

### LLMs

In [14]:
variables, years = get_variables(2000, 2022)

# Where to find data for ...
SBERT_LSC = "/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-bert-sentence-bert-swedish-cased.csv"
BERT_LSC  = "/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-bert-base-swedish-cased.csv"
MT5_LSC   = "/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-mt5-xl.csv"

SBERT_IOD = "/home/max/Results/replacements/results/fb_kb_results.csv"
BERT_IOD  = "/home/max/Results/replacements/results/fb_bert-kb-avg_results.csv"
MT5_IOD   = "/home/max/Results/replacements/results/fb_mt5-xl_results.csv"


# Sentence-BERT
fb_kb_lsc = get_df(SBERT_LSC, variables, terms)
fb_kb_iod = get_iod(SBERT_IOD, SBERT_LSC, variables, terms)

# BERT
fb_bert_lsc = get_df(BERT_LSC, variables, terms)
fb_bert_iod = get_iod(BERT_IOD, BERT_LSC, variables, terms)

# mT5
fb_mt5xl_lsc = get_df(MT5_LSC, variables, terms)
fb_mt5xl_iod = get_iod(MT5_IOD, MT5_LSC, variables, terms)

### SGNS

In [18]:
WS=5
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_lsc_w5 = get_df(f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}.csv", 
    variables_noRCH, terms, drop_frq = False)
fb_sgns_w5_iod = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w{WS}_results.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}.csv", 
    variables_noRCH, terms)
fb_sgns_w5_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

In [19]:
WS=10
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_lsc_w10 = get_df(f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}.csv", 
    variables_noRCH, terms, drop_frq = False)
fb_sgns_w10_iod = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w{WS}_results.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}.csv", 
    variables_noRCH, terms)
fb_sgns_w10_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

In [20]:
WS=15
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_lsc_w15 = get_df(f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}.csv", 
    variables_noRCH, terms, drop_frq = False)
fb_sgns_w15_iod = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w{WS}_results.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}.csv", 
    variables_noRCH, terms)
fb_sgns_w15_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

### 200 dimensions

In [21]:
WS=5
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_lsc_w5_200 = get_df(f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms, drop_frq = False)
fb_sgns_w5_200_iod = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w{WS}-200_results.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms)
fb_sgns_w5_200_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

In [22]:
WS=5
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_w5_200_iod_X = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w5-200-minimal_results_wsep.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms)
fb_sgns_w5_200_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

In [23]:
WS=10
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_lsc_w10_200 = get_df(f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms, drop_frq = False)
fb_sgns_w10_200_iod = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w{WS}-200_results.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms)
fb_sgns_w10_200_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

In [24]:
WS=15
variables_noRCH, _ = get_variables(2000, 2022, incl_rect=False)
fb_sgns_lsc_w15_200 = get_df(f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms, drop_frq = False)
fb_sgns_w15_200_iod = get_iod(f"/home/max/Results/replacments_w5-15_d100-200/results/fb_sgns-w{WS}-200_results.csv",
    f"/home/max/Documents/mlt/thesis/dw_results/fb_pol-yearly-radical3-restricted-w{WS}-200.csv", 
    variables_noRCH, terms)
fb_sgns_w15_200_iod.rename(columns={"B-strategy": "B-Strategy"}, inplace=True) 

### Add IOD difference

In [28]:
fb_kb_iod     = add_iod_dif(fb_kb_iod, years) # Sentence-BERT
fb_bert_iod     = add_iod_dif(fb_bert_iod, years)
fb_mt5xl_iod    = add_iod_dif(fb_mt5xl_iod, years)

fb_sgns_w5_iod  = add_iod_dif(fb_sgns_w5_iod, years)
fb_sgns_w10_iod = add_iod_dif(fb_sgns_w10_iod, years)
fb_sgns_w15_iod = add_iod_dif(fb_sgns_w15_iod, years)

fb_sgns_w5_200_iod   = add_iod_dif(fb_sgns_w5_200_iod,   years)
fb_sgns_w10_200_iod  = add_iod_dif(fb_sgns_w10_200_iod,  years)
fb_sgns_w15_200_iod  = add_iod_dif(fb_sgns_w15_200_iod,  years)

## Regression

In [ ]:
def data2data(
    iod_data, 
    lsc_data, 
    dwes, 
    AB_strat, 
    years, 
    methods, 
    baseline="first", 
    add_log_fpm=True, 
    add_freq_dif=True,
    LOG = np.log2
):
    
    df = iod_data.copy()
    
    df = df[df["DWE"].isin(dwes)]

    assert type(AB_strat) == dict
        
    for var, val in AB_strat.items():
        df = df[df[var]==val]
    
    df.drop(AB_strat.keys(), axis=1, inplace=True)
    
    transitions = [(year, str(int(year)+1)) for year in years[:-1]]
    
    lscs = ["fpm", "gch", "rch"]
    if add_freq_dif:
        lscs = lscs + ["fdf", "adf", "fpc", "apc"]
        lsc_data = lsc_data.copy()
        for yi, yj in transitions:
            lsc_data[f"fdf_{yi}:{yj}"] = lsc_data[f"fpm_{yj}"] - lsc_data[f"fpm_{yi}"] 
        for yi, yj in transitions:
            lsc_data[f"adf_{yi}:{yj}"] = abs(lsc_data[f"fpm_{yj}"] - lsc_data[f"fpm_{yi}"]) 
        for yi, yj in transitions:
            lsc_data[f"fpc_{yi}:{yj}"] = (lsc_data[f"fpm_{yj}"] - lsc_data[f"fpm_{yi}"]) / lsc_data[f"fpm_{yi}"]
        for yi, yj in transitions:
            lsc_data[f"apc_{yi}:{yj}"] = abs((lsc_data[f"fpm_{yj}"] - lsc_data[f"fpm_{yi}"]) / lsc_data[f"fpm_{yi}"])
        
    transitions = [f"{yi}:{yj}" for yi, yj in transitions]
    
    if baseline == "first":
        years = years[:-1]
    if baseline == "second":
        years = years[1:]
        
    new_df = []
    dw_yrs = []
    for method in methods:
        s    = df[df["Method"]==method][["DWE"]+years].set_index("DWE").sort_index()
        idx  = [dw for lst in [[dw]*len(years) for dw in s.index] for dw in lst]
        yrs  = years*len(s.index)
        dw_yrs.append(pd.Series([f"{dw}_{y}" for dw, y in zip(idx, yrs)], name=method))
        new_df.append(pd.Series(s.to_numpy().flatten(), name=method))
        
    for method in methods: 
        s   = df[df["Method"]==method][["DWE"]+transitions].set_index("DWE").sort_index()
        idx = [dw for lst in [[dw]*len(transitions) for dw in s.index] for dw in lst]
        trs  = transitions*len(s.index)
        dw_yrs.append(pd.Series([f"{dw}_{y}" for dw, y in zip(idx, trs)], name=method+"_dif"))
        new_df.append(pd.Series(s.to_numpy().flatten(), name=method+"_dif"))
    
    for lsc_measure in lscs:
        if lsc_measure == "fpm":
            variables = [col for col in lsc_data.columns if col.startswith("fpm") and col.split("_")[-1] in years]
            s = lsc_data.loc[dwes][variables].sort_index()
            idx  = [dw for lst in [[dw]*len(years) for dw in s.index] for dw in lst]
            yrs  = years*len(s.index)
            dw_yrs.append(pd.Series([f"{dw}_{y}" for dw, y in zip(idx, yrs)], name=lsc_measure))
            new_df.append(pd.Series(s.to_numpy().flatten(), name=lsc_measure))
            if add_log_fpm:
                new_df.append(pd.Series(LOG(s.to_numpy().flatten()), name=lsc_measure+"_log"))
            
        else:
            variables = [col for col in lsc_data.columns if col.startswith(lsc_measure)]
            s = lsc_data.loc[dwes][variables].sort_index()
            idx = [dw for lst in [[dw]*len(transitions) for dw in s.index] for dw in lst]
            trs  = trs*len(s.index)
            dw_yrs.append(pd.Series([f"{dw}_{y}" for dw, y in zip(idx, trs)], name=lsc_measure))
            new_df.append(pd.Series(s.to_numpy().flatten(), name=lsc_measure))
    
    check = pd.concat(dw_yrs, axis=1)
    
    if False in (check[methods].eq(check["fpm"], axis=0)).all(axis=1):
        print("No match")
    
    if False in (check[[col for col in check.columns if col.endswith("dif")]].eq(check[["gch","rch"]], axis=0)).all(axis=1):
        print("No match")
        
    data = pd.concat(new_df+[check["fpm"].rename("DW-YR")], axis=1)
    
    return data


In [ ]:
def OLS(data, DEP, INDEP, transform_dep=False, dep_transformer=None, LOG=np.log2): 
    
    df = data[[DEP]+INDEP].copy()
    df = df.dropna(axis=0)
    
    X = df[INDEP]
    y = df[DEP]
    
    if transform_dep: 
        if dep_transformer == None:
            y = LOG(y)
        elif dep_transformer == "rank":
            y = rankdata(y)
        else:
            y = y.values.reshape(-1,1)
            if dep_transformer == "box-cox":
                transformer = PowerTransformer(method='box-cox')
                y = transformer.fit_transform(y)
                print("Lambda = ", transformer.lambdas_[0])
            elif dep_transformer == "yeo-johnson":
                y = power_transform(y, method='yeo-johnson')
            else:
                transformer = dep_transformer
                y = transformer.fit_transform(y)

    X = sm.add_constant(X)

    model = sm.OLS(y, X, missing="drop").fit()

    return model

In [ ]:
def revise_data(data, indep, dep, iod_var, log_iod=False, LOG=np.log2):
    
    data = data[indep+[dep]]
    data = data.dropna()
    data[iod_var] = abs(data[iod_var])
    if log_iod:
        data[iod_var] = LOG(data[iod_var])
    return data

In [ ]:
def VIF(data, dep):
    exog_data_const = sm.add_constant(data[[c for c in data.columns if c != dep]])
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(exog_data_const.values, i) for i in range(exog_data_const.shape[1])]
    vif = vif.transpose()
    vif.columns = exog_data_const.columns
    return vif

In [ ]:
def test_normality_multi( # test assumptions
    names, # list
    iods, # list
    lscs, # list
    dep, 
    indep, 
    iod_var, 
    dwes, 
    abstrats, # list 
    years, 
    methods, 
    baseline, 
    log_transform,
    log_transform_iod = False,
    test_stat = "jarque_bera", 
    LOG=np.log2
):
    resids = []
    VIFs = []
    
    if iod_var not in indep:
        indep.append(iod_var)
    
    for iod, lsc, abstrat in zip(iods, lscs, abstrats):
    
        data = data2data(
            iod_data = iod, 
            lsc_data = lsc, 
            dwes     = dwes,
            AB_strat = abstrat, 
            years    = years,
            methods  = methods,
            baseline = baseline
            )
        
        data = revise_data(data, indep, dep, iod_var, log_iod=log_transform_iod, LOG=LOG)
        
        vif = VIF(data, dep)
        VIFs.append(vif)

        res = OLS(data, dep, indep, transform_dep=log_transform, LOG=LOG)#, dep_transformer="box-cox" )
        resids.append(res.resid)
    
    dfs = []
    for r in resids:
        normality_res = pg.normality(r, method=test_stat)
        dfs.append(normality_res)
        
    df = pd.concat(dfs, axis = 0)
    df.index = names
    VIF_tab = pd.concat(VIFs)
    VIF_tab.index = names
    
    return df, VIF_tab
        

In [ ]:
def stargazer_multi(
    names, # list
    iods, # list
    lscs, # list
    dep, 
    indep, 
    iod_var, 
    dwes, 
    abstrats, # list 
    years, 
    methods, 
    baseline, 
    log_transform,
    log_transform_iod = False,
    #test_stat = "jarque_bera",
    LOG=np.log2,
    norm=False
):
    models = []
    
    if iod_var not in indep:
        indep.append(iod_var)
    
    for iod, lsc, abstrat in zip(iods, lscs, abstrats):
    
        data = data2data(
            iod_data = iod, 
            lsc_data = lsc, 
            dwes     = dwes,
            AB_strat = abstrat, 
            years    = years,
            methods  = methods,
            baseline = baseline,
            LOG=LOG
            )
        
        data = revise_data(data, indep, dep, iod_var)
        if log_transform:
            data[dep] = LOG(data[dep])
        if log_transform_iod:
            data[iod_var] = LOG(data[iod_var])        
                
        if norm:
            for c in data.columns:
                data[c] = zscore(data[c])
            
        res = OLS(data, dep, indep)
        models.append(res)
    
    ziggy = Stargazer(models)
    ziggy.custom_columns(names)#, [1, 1])
    ziggy.show_model_numbers(False)
    ziggy.covariate_order(sorted(indep)+["const"])
    ziggy.rename_covariates({'cnt-ssc_dif': '$\Delta ^{IOR}$', 'fpc':'$\Delta ^{FPM}$', 'fpm_log': 'FPM (log)' })
    ziggy.significance_levels([0.05, 0.01, 0.001])
    
    return ziggy
        

## Analysis

In [ ]:
ALL_FB_LSC = [
fb_kb_lsc,
fb_bert_lsc,
fb_mt5xl_lsc,
fb_sgns_lsc_w5,
fb_sgns_lsc_w10,
fb_sgns_lsc_w15,
fb_sgns_lsc_w5_200,
fb_sgns_lsc_w10_200,
fb_sgns_lsc_w15_200    
]

In [ ]:
ALL_FB_IOD = [
fb_kb_iod,
fb_bert_iod,
fb_mt5xl_iod,
fb_sgns_w5_iod,
fb_sgns_w10_iod,
fb_sgns_w15_iod,
fb_sgns_w5_200_iod,
fb_sgns_w10_200_iod,
fb_sgns_w15_200_iod
]

In [ ]:
names = [
"SBERT", #"fb_kb",
"BERT", #"fb_bert",
"mT5-XL",
"SGNS-w5",
"SGNS-w10",
"SGNS-w15",
"SGNS-w5-200",
"SGNS-w10-200",
"SGNS-w15-200"
]

In [ ]:
# Slection strategies
ABSTRATs = [{"A-Strategy":"rn"}] * 3 + [{"A-Strategy":"ms1", "B-Strategy":"min0.2"}] * 6

In [ ]:
DEP_TRANSFORM = False
IOD_TRANSFORM = False
IOD_VAR = "cnt-ssc_dif"
INDEP = [
    IOD_VAR,
    "fpc", 
#     "fpm"
    "fpm_log"
]
LOG = np.log2

In [ ]:
table, vif = test_normality_multi(
    names = names, # list
    iods = ALL_FB_IOD, # list
    lscs = ALL_FB_LSC, # list
    dep = "gch", 
    indep = INDEP, 
    iod_var = IOD_VAR, 
    dwes = ['V1_berika', 'N1_globalist', 'N1_återvandring', 'N1_förortsgäng'], 
    abstrats = ABSTRATs, # list 
    years = years, 
    methods = ["I-cnt", "O-cnt", "cnt-ssc", "cnt-smx"], 
    baseline = "first", 
    log_transform = DEP_TRANSFORM,
    log_transform_iod=IOD_TRANSFORM,
#     test_stat = "shapiro" # "jarque_bera" (default),
    LOG = LOG
)
table.round(3)

In [ ]:
vif

In [ ]:
# print(table.to_latex(float_format="{:0.3f}".format))

In [ ]:
all_stars = stargazer_multi(
    names = names, # list
    iods = ALL_FB_IOD, # list
    lscs = ALL_FB_LSC, # list
    dep = "gch", 
    indep = INDEP, 
    iod_var = IOD_VAR, 
    dwes = ['V1_berika', 'N1_globalist', 'N1_återvandring', 'N1_förortsgäng'], 
    abstrats = ABSTRATs, # list 
    years = years, 
    methods = ["I-cnt", "O-cnt", "cnt-ssc", "cnt-smx"], 
    baseline = "first", 
    log_transform = DEP_TRANSFORM,
    log_transform_iod=IOD_TRANSFORM,
#     test_stat = "shapiro" # "jarque_bera" (default),
    LOG=LOG,
    norm = True
); all_stars

In [ ]:
print(all_stars.render_latex())